In [54]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os.path
import re
import sys
import tarfile

from six.moves import urllib
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy as sp
import random

sns.set(style="ticks", color_codes=True)
%matplotlib inline

In [56]:
def maybe_download_and_extract():
  """Download and extract model tar file."""
  dest_directory = FLAGS.model_dir
  if not os.path.exists(dest_directory):
    os.makedirs(dest_directory)
  filename = DATA_URL.split('/')[-1]
  filepath = os.path.join(dest_directory, filename)
  if not os.path.exists(filepath):
    def _progress(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s %.1f%%' % (
          filename, float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()
    filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
  tarfile.open(filepath, 'r:gz').extractall(dest_directory)

In [57]:
maybe_download_and_extract()

In [3]:
FLAGS = None

# pylint: disable=line-too-long
DATA_URL = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
# pylint: enable=line-too-long


In [4]:
class NodeLookup(object):
  """Converts integer node ID's to human readable labels."""

  def __init__(self,
               label_lookup_path=None,
               uid_lookup_path=None):
    if not label_lookup_path:
      label_lookup_path = os.path.join(
          FLAGS.model_dir, 'imagenet_2012_challenge_label_map_proto.pbtxt')
    if not uid_lookup_path:
      uid_lookup_path = os.path.join(
          FLAGS.model_dir, 'imagenet_synset_to_human_label_map.txt')
    self.node_lookup = self.load(label_lookup_path, uid_lookup_path)

  def load(self, label_lookup_path, uid_lookup_path):
    """Loads a human readable English name for each softmax node.

    Args:
      label_lookup_path: string UID to integer node ID.
      uid_lookup_path: string UID to human-readable string.

    Returns:
      dict from integer node ID to human-readable string.
    """
    if not tf.gfile.Exists(uid_lookup_path):
      tf.logging.fatal('File does not exist %s', uid_lookup_path)
    if not tf.gfile.Exists(label_lookup_path):
      tf.logging.fatal('File does not exist %s', label_lookup_path)

    # Loads mapping from string UID to human-readable string
    proto_as_ascii_lines = tf.gfile.GFile(uid_lookup_path).readlines()
    uid_to_human = {}
    p = re.compile(r'[n\d]*[ \S,]*')
    for line in proto_as_ascii_lines:
      parsed_items = p.findall(line)
      uid = parsed_items[0]
      human_string = parsed_items[2]
      uid_to_human[uid] = human_string

    # Loads mapping from string UID to integer node ID.
    node_id_to_uid = {}
    proto_as_ascii = tf.gfile.GFile(label_lookup_path).readlines()
    for line in proto_as_ascii:
      if line.startswith('  target_class:'):
        target_class = int(line.split(': ')[1])
      if line.startswith('  target_class_string:'):
        target_class_string = line.split(': ')[1]
        node_id_to_uid[target_class] = target_class_string[1:-2]

    # Loads the final mapping of integer node ID to human-readable string
    node_id_to_name = {}
    for key, val in node_id_to_uid.items():
      if val not in uid_to_human:
        tf.logging.fatal('Failed to locate: %s', val)
      name = uid_to_human[val]
      node_id_to_name[key] = name

    return node_id_to_name

  def id_to_string(self, node_id):
    if node_id not in self.node_lookup:
      return ''
    return self.node_lookup[node_id]


def create_graph():
  """Creates a graph from saved GraphDef file and returns a saver."""
  # Creates graph from saved graph_def.pb.
  with tf.gfile.FastGFile(os.path.join(
      FLAGS.model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')


def run_inference_on_image(images):
  """Runs inference on an image.

  Args:
    image: Image file name.

  Returns:
    Nothing
  """

  res = []
  with tf.Session() as sess:
      for image in images:
        if not tf.gfile.Exists(image):
          tf.logging.fatal('File does not exist %s', image)
        image_data = tf.gfile.FastGFile(image, 'rb').read()

        # Creates graph from saved GraphDef.
        create_graph()
        softmax_tensor = sess.graph.get_tensor_by_name('softmax:0')
        pool_tensor = sess.graph.get_tensor_by_name('pool_3:0')

        # Creates graph from saved GraphDef.
        print("create graph start")
        create_graph()
        print("create graph end")
        
        # Some useful tensors:
        # 'softmax:0': A tensor containing the normalized prediction across
        #   1000 labels.
        # 'pool_3:0': A tensor containing the next-to-last layer containing 2048
        #   float description of the image.
        # 'DecodeJpeg/contents:0': A tensor containing a string providing JPEG
        #   encoding of the image.
        # Runs the softmax tensor by feeding the image_data as input to the graph.
        print("prediction start")
        predictions = sess.run(pool_tensor, {'DecodeJpeg/contents:0': image_data})
        print("prediction end")
        predictions = np.squeeze(predictions)
        res.append(predictions)
  return res

    # Creates node ID --> English string lookup.
#     node_lookup = NodeLookup()

#     top_k = predictions.argsort()[-FLAGS.num_top_predictions:][::-1]
#     for node_id in top_k:
#       human_string = node_lookup.id_to_string(node_id)
#       score = predictions[node_id]
#       print('%s (score = %.5f)' % (human_string, score))

In [5]:
parser = argparse.ArgumentParser()
# classify_image_graph_def.pb:
#   Binary representation of the GraphDef protocol buffer.
# imagenet_synset_to_human_label_map.txt:
#   Map from synset ID to a human readable string.
# imagenet_2012_challenge_label_map_proto.pbtxt:
#   Text representation of a protocol buffer mapping a label to synset ID.
parser.add_argument(
  '--model_dir',
  type=str,
  default='/tmp/imagenet',
  help="""\
  Path to classify_image_graph_def.pb,
  imagenet_synset_to_human_label_map.txt, and
  imagenet_2012_challenge_label_map_proto.pbtxt.\
  """
)
parser.add_argument(
  '--image_file',
  type=str,
  default='',
  help='Absolute path to image file.'
)
parser.add_argument(
  '--num_top_predictions',
  type=int,
  default=5,
  help='Display this many predictions.'
)
FLAGS, unparsed = parser.parse_known_args()
# tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

In [6]:
def main():
  image = "/Users/gvsi/Dropbox/Photos/California Road Trip/2017-01-01 01.17.42.jpg"
  run_inference_on_image(image)

In [7]:
# tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

In [8]:
data = train=pd.read_csv('dataset.csv', sep=',', index_col=False)

In [17]:
# run_inference_on_image("img/" + data['alias'][0] + "/" + data['filename'][0])
# "img/" + data['alias'][0] + "/" + data['filename'][0]

images = ["img/" + data['alias'][0] + "/" + data['filename'][0], "img/" + data['alias'][1] + "/" + data['filename'][1],
         "img/" + data['alias'][2] + "/" + data['filename'][2], "img/" + data['alias'][3] + "/" + data['filename'][3]]

res = run_inference_on_image(images)

create graph start
create graph end
prediction start
prediction end
create graph start
create graph end
prediction start
prediction end
create graph start
create graph end
prediction start
prediction end
create graph start
create graph end
prediction start
prediction end


In [13]:
images

['img/yarasantosoficial/18094753_827986990700746_4918745179017445376_n.jpg',
 'img/yarasantosoficial/18096144_1889303301327976_1636725699073015808_n.jpg',
 'img/yarasantosoficial/18095912_1359632760784016_7285237765015339008_n.jpg',
 'img/yarasantosoficial/18013277_1319178868174444_2724993046633512960_n.jpg']

In [20]:
res[0].shape

(2048,)

In [11]:
paths = "img/" + data['alias'] + "/" + data['filename']

In [12]:
paths[0:1000]

0      img/yarasantosoficial/18094753_827986990700746...
1      img/yarasantosoficial/18096144_188930330132797...
2      img/yarasantosoficial/18095912_135963276078401...
3      img/yarasantosoficial/18013277_131917886817444...
4      img/yarasantosoficial/18012095_415218078850933...
5      img/yarasantosoficial/18013411_294597020963076...
6      img/yarasantosoficial/['17881034_1519486784742...
7      img/yarasantosoficial/18011556_117461155473597...
8      img/yarasantosoficial/17934022_294123917691342...
9      img/yarasantosoficial/17932721_607294366141772...
10     img/londonfoodbabes/17932077_1296914350425715_...
11     img/londonfoodbabes/18096429_683922238458096_5...
12     img/londonfoodbabes/18160936_420622264979454_5...
13     img/londonfoodbabes/18160698_464226950580386_5...
14     img/londonfoodbabes/18013813_948212631948663_6...
15     img/londonfoodbabes/18160249_695951757273842_7...
16     img/londonfoodbabes/18095941_428212000874072_5...
17     img/londonfoodbabes/1801

In [41]:
res = np.zeros((1000, 2049))

In [42]:
np.ones((1, 2048)).shape

(1, 2048)

In [43]:
# res[0, 1:2049] = np.ones((1, 2048))

In [46]:
# res[:,0] = np.arange(1000)

In [53]:
res

array([[   0.,    1.,    1., ...,    1.,    1.,    1.],
       [   1.,    0.,    0., ...,    0.,    0.,    0.],
       [   2.,    0.,    0., ...,    0.,    0.,    0.],
       ..., 
       [ 997.,    0.,    0., ...,    0.,    0.,    0.],
       [ 998.,    0.,    0., ...,    0.,    0.,    0.],
       [ 999.,    0.,    0., ...,    0.,    0.,    0.]])

In [52]:
df = pd.DataFrame(res)

In [ ]:
df.to_csv()